### 一、读取csv文件以及保存csv文件

In [68]:
import pandas as pd
train = pd.read_csv('train.csv') 
train_small = train.head(80000) 
train_small.to_csv('train_small.csv',index=False)   ## index这个参数一般设置为false，如果不想要列名可以使用header参数设置

### 二、查看对象的数据类型

In [10]:
type(train_small) 

pandas.core.frame.DataFrame

### 三、查看前几行

In [11]:
train_small.head(5) 

,orderid,userid,bikeid,biketype,starttime,geohashed_start_loc,geohashed_end_loc
0,1893973,451147,210617,2,2017-05-14 22:16:50,wx4snhx,wx4snhj
1,4657992,1061133,465394,1,2017-05-14 22:16:52,wx4dr59,wx4dquz
2,2965085,549189,310572,1,2017-05-14 22:16:51,wx4fgur,wx4fu5n
3,4548579,489720,456688,1,2017-05-14 22:16:51,wx4d5r5,wx4d5r4
4,3936364,467449,403224,1,2017-05-14 22:16:50,wx4g27p,wx4g266


### 四、转化时间字段&增添新的字段&map映射的使用

In [17]:
train_small['hour'] = pd.to_datetime(train_small['starttime']).map(lambda x:x.strftime('%H')) 
train_small['hour_duan'] = train_small['hour'].map(hourDuan)

C:\Users\lenovo\python\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\lenovo\python\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [16]:
def hourDuan(hour):
    hour = int(hour)
    hotHourList1 = [7,8,9,12,13,14,18,19,20,21]
    hotHourList2 = [12, 13, 14]
    hotHourList3 = [18, 19, 20, 21]
    if hour in hotHourList1:
        return 1
    elif hour in hotHourList2:
        return 2
    elif hour in hotHourList3:
        return 3
    else:
        return 0

In [18]:
train_small.head(5)  

,orderid,userid,bikeid,biketype,starttime,geohashed_start_loc,geohashed_end_loc,hour,hour_duan
0,1893973,451147,210617,2,2017-05-14 22:16:50,wx4snhx,wx4snhj,22,0
1,4657992,1061133,465394,1,2017-05-14 22:16:52,wx4dr59,wx4dquz,22,0
2,2965085,549189,310572,1,2017-05-14 22:16:51,wx4fgur,wx4fu5n,22,0
3,4548579,489720,456688,1,2017-05-14 22:16:51,wx4d5r5,wx4d5r4,22,0
4,3936364,467449,403224,1,2017-05-14 22:16:50,wx4g27p,wx4g266,22,0


### 五、groupby的使用：说白了就是在groupby后面跟的字段后面再添一个新的字段

#### &emsp;&emsp;groupby一共分为split，apply，combine几步，首先按照某一字段划分成不同的块split，在不同的块中进行操作apply，最后combine
#### 经常使用的是df.groupby(['Name'],as_index=False).apply(methodName)或df.groupby(['Name'],as_index=False)
#### ['Name2'].agg{'Name3':'preMethod'},其中Name*是字段名称，methodName是自定义的method，preMethod是系统预定义好的method，

#### 包括count,size,nunique,里面的count和size都是统计记录数，而nunique是统计不同的个数。上面相关使用的demo如下。

## 方式一：agg 

#### &emsp;&emsp;Tip: 对于count这种preMethod，Name2实际是没有用的，但是对于nunique用于统计Name2不同的个数的时候却能够起到作用。除此之外，
#### 这样的使用方法返回的是一个dataframe，这个dataframe的字段只包含两个部分，Name1，Name3。

In [24]:
user_count = train_small.groupby(['userid'],as_index=False)['geohashed_end_loc'].agg({'user_count':'count'})
user_end_count = train_small.groupby(['userid','geohashed_end_loc'],as_index=False)['geohashed_end_loc'].agg({'user_end_count':'count'}) 

In [22]:
user_count.head(1)

,userid,user_count
0,3,1


In [23]:
user_end_count.head(1) 

,userid,geohashed_end_loc,user_end_count
0,3,wx4en7t,1


In [28]:
UniqueLocNum = train_small.groupby(['userid','geohashed_start_loc'],as_index=False)['geohashed_end_loc'].agg({'unique_eloc':'nunique'})   
# 统计一个start_loc有几个不同的end_loc
UniqueLocNum = UniqueLocNum.reset_index() 
# 在上一步完成之后一般都需要reset一下

In [29]:
UniqueLocNum.head(2)

,index,userid,geohashed_start_loc,unique_eloc
0,0,3,wx4ene7,1
1,1,17,wx4gd0m,1


## 方式二：apply(freeMethod)
### 产生一个新的字段，以userid为划分依据，biketype+bikeid的值

In [31]:
def btPlusbi(group):
    group['bpb'] = group['biketype']+group['bikeid']
    return group[['userid','bpb']]
bPb = train_small.groupby(['userid']).apply(btPlusbi) 

In [33]:
bPb.head(2) 

,userid,bpb
0,451147,210619
1,1061133,465395


### 有的时候还可以把单独的字段转成list之后再做处理

In [34]:
def help_bike_leak(group):
    start_loc = list(group['geohashed_start_loc'])
    end_loc = list(group['geohashed_end_loc'])
    time = list(group['starttime'])
    newFieldList = []
    group['newField'] = newFieldList

### 六、merge与concat

### merge

In [37]:
train_small.head(1) 

,orderid,userid,bikeid,biketype,starttime,geohashed_start_loc,geohashed_end_loc,hour,hour_duan
0,1893973,451147,210617,2,2017-05-14 22:16:50,wx4snhx,wx4snhj,22,0


In [38]:
user_count.head(1)

,userid,user_count
0,3,1


In [39]:
result = pd.merge(train_small,user_count,on=['userid'],how='left')   # 以userid进行左连接，当然可以选取多个字段作为连接标准

In [40]:
result.head(2)

,orderid,userid,bikeid,biketype,starttime,geohashed_start_loc,geohashed_end_loc,hour,hour_duan,user_count
0,1893973,451147,210617,2,2017-05-14 22:16:50,wx4snhx,wx4snhj,22,0,1
1,4657992,1061133,465394,1,2017-05-14 22:16:52,wx4dr59,wx4dquz,22,0,3


### concat

In [43]:
df1 = pd.DataFrame([['a', 1], ['b', 2]],columns=['letter', 'number'])
df2 = pd.DataFrame([['c', 3], ['d', 4]],columns=['letter', 'number'])
df3 = pd.DataFrame([['c', 3, 'cat'], ['d', 4, 'dog']],columns=['letter', 'number', 'animal'])
r1 = pd.concat([df1,df2])
r2 = pd.concat([df1,df3],ignore_index=True) 

In [44]:
r1

,letter,number
0,a,1
1,b,2
0,c,3
1,d,4


In [45]:
r2

,animal,letter,number
0,NaN,a,1
1,NaN,b,2
2,cat,c,3
3,dog,d,4


### 七、使用apply函数对某个字段做数学变换并且使用astype改变某个字段的数据类型

In [46]:
train_small.head(1) 

,orderid,userid,bikeid,biketype,starttime,geohashed_start_loc,geohashed_end_loc,hour,hour_duan
0,1893973,451147,210617,2,2017-05-14 22:16:50,wx4snhx,wx4snhj,22,0


In [70]:
train_small['month'] = train_small['starttime'].apply(lambda x:x.split()[0].split('-')[1]).astype(int) 

C:\Users\lenovo\python\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [49]:
train_small.head(1) 

,orderid,userid,bikeid,biketype,starttime,geohashed_start_loc,geohashed_end_loc,hour,hour_duan,month
0,1893973,451147,210617,2,2017-05-14 22:16:50,wx4snhx,wx4snhj,22,0,5


### 八、使用浅复制df.copy()

### 九、把dataframe变成np.array

In [59]:
train_copy = train_small.copy()
train_array = train_copy.values
type(train_array) 

numpy.ndarray

### 十、df.drop()一次性删除dataframe中的多个字段以及del df['fieldName']删除一个字段

In [52]:
train_copy.head(2)

,orderid,userid,bikeid,biketype,starttime,geohashed_start_loc,geohashed_end_loc,hour,hour_duan,month
0,1893973,451147,210617,2,2017-05-14 22:16:50,wx4snhx,wx4snhj,22,0,5
1,4657992,1061133,465394,1,2017-05-14 22:16:52,wx4dr59,wx4dquz,22,0,5


In [60]:
train_copy.drop(['orderid','bikeid'],axis=1).head(2) 

,userid,biketype,starttime,geohashed_start_loc,geohashed_end_loc,hour,hour_duan,month
0,451147,2,2017-05-14 22:16:50,wx4snhx,wx4snhj,22,0,5
1,1061133,1,2017-05-14 22:16:52,wx4dr59,wx4dquz,22,0,5


In [61]:
del train_copy['month']  ## 由于之前的是浅复制，所以上面的两个字段并没有被删除，需要指定inplace参数

In [62]:
train_copy.head(2)

,orderid,userid,bikeid,biketype,starttime,geohashed_start_loc,geohashed_end_loc,hour,hour_duan
0,1893973,451147,210617,2,2017-05-14 22:16:50,wx4snhx,wx4snhj,22,0
1,4657992,1061133,465394,1,2017-05-14 22:16:52,wx4dr59,wx4dquz,22,0


### 十一、过滤符合条件的dataframe

In [63]:
train_small.head(2) 

,orderid,userid,bikeid,biketype,starttime,geohashed_start_loc,geohashed_end_loc,hour,hour_duan,month
0,1893973,451147,210617,2,2017-05-14 22:16:50,wx4snhx,wx4snhj,22,0,5
1,4657992,1061133,465394,1,2017-05-14 22:16:52,wx4dr59,wx4dquz,22,0,5


In [64]:
train_small = train_small[train['biketype']==2]

C:\Users\lenovo\python\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [65]:
train_small.head(2)

,orderid,userid,bikeid,biketype,starttime,geohashed_start_loc,geohashed_end_loc,hour,hour_duan,month
0,1893973,451147,210617,2,2017-05-14 22:16:50,wx4snhx,wx4snhj,22,0,5
8,2803108,15906,295614,2,2017-05-14 22:16:55,wx4f8t9,wx4f8tj,22,0,5


### 十二、使用pd.get_dummies对df的某个字段做编码,还可以对series,array进行编码

In [71]:
month_onehot = pd.get_dummies(train_small['month'],prefix='onehot') 

In [73]:
month_onehot.head(2)  ## 由于train_small里面只有5月的数据，所以只有这一个onehot之后的字段

,onehot_5
0,1
1,1


### 十三、根据某些字段去重df.drop_duplicates()

In [74]:
train_small.head(2)

,orderid,userid,bikeid,biketype,starttime,geohashed_start_loc,geohashed_end_loc,month
0,1893973,451147,210617,2,2017-05-14 22:16:50,wx4snhx,wx4snhj,5
1,4657992,1061133,465394,1,2017-05-14 22:16:52,wx4dr59,wx4dquz,5


In [75]:
train_small.drop_duplicates(['userid','bikeid'],inplace=True)

C:\Users\lenovo\python\lib\site-packages\pandas\util\decorators.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)


### 十四、通过dataframe获取到feature和label对应的array形式数据

In [77]:
##  假若上面的train_small数据中，label是month，剩余的字段对应的是features
X = train_small.values[:,:-1]
y = train_small.values[:,-1] 